# Project Title
### Data Engineering Capstone Project

#### Project Summary

USA is the land of opportunities, and one of most (if not the most) required destinations of immigrants.
year after year, USA is providing opportunities like lottery of green card, work permits and students visa.

One of demands of american governments is regulating and tuning the immigration according to USA needs.

Project in hands is collecting related data sets into a simple data ware house where analysts can investigate and come up with insights about immigration and different factors those can affect it.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DateType
from pyspark.sql.functions import expr , udf ,trim ,year, month, dayofmonth

from datetime import datetime
import matplotlib.pyplot as plt
import configparser

# CONFIG
config = configparser.ConfigParser()
config.read('dwh.cfg')
DWH_ROLE_ARN           = config.get('IAM_ROLE','ARN')
S3_bucket              = config.get('S3','S3_bucket')
accesskey              = config.get('S3','accesskey')
secretkey              = config.get('S3','secretkey')

###### Step 1: Scope the Project and Gather Data

#### Scope 

Current project is dealing mainly with immigration data comming from USA immigration data set as primary source of data beside 3 other data sets including temprature, cities and airports.
there are also small dictionary tables found will help us to explain data, like countries and states.

#### Plan 
there are many alternatives for processing data in this project. 2 of them will be explained in details

1. Spark data frames and Spark SQL : 
    1. reading system files (csv and parquet) using Spark Data Frame
    2. Generating data files from existing dictionary attached.
    3. Data Quality using Spark Data frame and Spark SQL
    4. Generating star model schema from data sets and dictionaries.
    5. Storing new generated schema to csv or parquet files.
    
    
2. S3 and Redshift :
    1. Uploading data to S3 bucket
    2. Loading data from S3 to Redshift.
    3. Data Quality for raw data sets.
    4. Generating Star Model for new tables in redshift
    5. Data Quality on generated data in new tables.   


#### Describe and Gather Data 
   
1. I94 Immigration Data : data set coming from **US National Tourism and Trade Office** it contains data about immigrants {like year of birth, nationality, ..} beside their visa info {arrival, departure, type ...} and which airport they came through.

2. World Temperature Data: This dataset came from **Kaggle** , it includes average temprature for city over number of years.

3. U.S. City Demographic Data: from **OpenSoft**. it contains city name, state in addition to some statistics about its population.

4. Airport Code Table: a simple table of airport codes, it can be generated from **IATA** or other sites related to flight regulations.


| Data Set | Size in MB |  Number of rows |
|----------|:-------------:|------:|
| Immigration |  6000 | nearly 1,600,000  |
| Temprature |  509|  8,599,213|  
| Airports |  6|  55,075| 
| Cities |  0.25|  2,891| 

in addition to dictionaries from attached files

| Data Set | Data Set  |  Columns |
|----------|:-------------:|------:|
| Countries |  Immigration  | i94cit, i94res  |
| states |  Immigration and airports|  i94addr , state|  
| ports |  immigration |  i94port| 
| Visa model |  immigration|  i94mod| 
| Visa  |  immigration|  i94visa|

#### Use Cases
resulting data model will be supporting data analytics team to answer many questions like the following

1. what is the ratio of different types of visas granted (student , pleasure , Work ) ?
2. which countries are the biggest source of immigrants?
3. the ratio of Foreign-born for every city / state, and which states have the biggest ratio of foreign-born vs. locals?
4. which states are preferred for immigrants (student , work , pleasure) ?
5. what are the range of ages for immigrants ?
    

In [2]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [3]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [1]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")
df_spark.createOrReplaceTempView('immigrationraw')
spark.sql(''' select * from immigrationraw limit 10''').show()

### Step 2: Explore and Assess the Data
#### Explore the Data 

raw data sets are including a lot of issues but important data still could be retrieved from it.

1. Immigration Data set includes the following issues:
    1. many columns contain null values, but most important are:
        occup: the occupation of immigrant
        i94addr: the state where immigrant will reside.
        gender
        
        while other columns are logical to have null, like DEPTDATE, it's logical if immigrant still inside USA , or even if he didn't leave and stayed illegally.
        
    2. this data set contains many formats of dates, one of them is **SAS date** format, which is the number of days since 1/1/1960
        some date formats are **yyyyMMdd** and some are **MMddyyyy** which is strange to be in the same data set.
    3. a lot of codes those are meaning less, and needs to be interpreted with the attached dictionary, like **i94cit** which is the citizenship of immigrants.
        and **i94res** which is the source country of issuing visa.
        
2. Airports: although should be accurate, it contains a lot of nulls in **iata_code**, this is why it can't match except 251 value of 299 airport existing in immigration data set
3. Cities : sounds very organized data set , although the summation of different races counts are greated than the total number for most of cities.
4. Temprature: very huge data set contains temprature degrees since before 1800 which is useless to our case, so data will be filtered based on date.
    
#### Cleaning Steps

main data set is immigration, so most of effort was to clean this set and make it readable.
1- SAS date was transformed to date type
    ```
    immigrationDf = immigrationDf.withColumn('arrdate2', expr("date_add(to_date('1960-01-01'),arrdate)"))
    immigrationDf = immigrationDf.withColumn('depdate2', expr("date_add(to_date('1960-01-01'),depdate)"))
    ```
2- String columns with different date formats are transferred to Date
    ```
    immigrationDf = immigrationDf.withColumn('dtadfile2', expr("to_date(dtadfile ,'yyyyMMdd')"))
    immigrationDf = immigrationDf.withColumn('dtaddto2', expr("to_date(dtaddto,'MMddyyyy')"))
    ```
3- for ambiguous columns, all related data are retrieved, dictionary tables are created to interpret vague columns like 
    i94res, i94cit , i94mod, i94visa
4- Dictionary files contains a lot of problems where it didn't comply directly to schema because of missing separators, or extra separators 
    a lot of manual corrections done to extract data to be useful in our data model.
    final dictionary files are attached in dictionary folder.
5- **biggest problem** in data of airports are not matching completely, from 299 airport mentioned in immigration data set, only 251 matching airport in airports data set by iata_code. and not all cities are matching with cities data set.
i tried with ports dictionary, although it matched 100% of airports as iata_code but a lot of airports are missing the cities.

so i used external data source from iata, i had in data analysis nano degree project.

    

### Step 3: Define the Data Model
#### Before data modeling
from attached SAS_Labes_description, i generated 5 csv files, all of them will be used while pipelining raw data into formatted star model schema.
these files are :
1. countries.csv
2. states.csv
3. ports.csv
4. visaModel.csv
5. visa.csv

the file createRawSchema.sql can be used to create raw tables in Redshift.

and the following graph represents the schema of raw data sets beside dictionary tables those helping interpret data.
<img src="graphs/rawSchema.png">

the below code is used to transfer file from S3 to redshift raw schema after uploading data files to s3bucket

In [ ]:
copy tempratureraw from S3_bucket+'GlobalLandTemperaturesByCity.csv'
ACCESS_KEY_ID accesskey
SECRET_ACCESS_KEY secretkey
format as csv 
IGNOREHEADER 1;

copy citiesraw from S3_bucket+'us-cities-demographics.csv'
ACCESS_KEY_ID accesskey
SECRET_ACCESS_KEY secretkey
format as csv 
IGNOREHEADER 1 delimiter ';';

copy airportsraw from S3_bucket+'airport-codes_csv.csv'
ACCESS_KEY_ID accesskey
SECRET_ACCESS_KEY secretkey
format as csv 
IGNOREHEADER 1;

copy immigrationraw from S3_bucket+'immigrationparquet'
iam_role DWH_ROLE_ARN
format as parquet ;

copy countriesdic from S3_bucket+'countries.csv'
ACCESS_KEY_ID accesskey
SECRET_ACCESS_KEY secretkey
format as csv 
IGNOREHEADER 1;

copy statesdic from S3_bucket+'states.csv'
ACCESS_KEY_ID accesskey
SECRET_ACCESS_KEY secretkey
format as csv 
IGNOREHEADER 1;

copy portsdic from S3_bucket+'ports.csv'
ACCESS_KEY_ID accesskey
SECRET_ACCESS_KEY secretkey
format as csv 
IGNOREHEADER 1;

copy visamodeldic from S3_bucket+'VisaModel.csv'
ACCESS_KEY_ID accesskey
SECRET_ACCESS_KEY secretkey
format as csv 
IGNOREHEADER 1;

copy visadic from S3_bucket+'visa.csv'
ACCESS_KEY_ID accesskey
SECRET_ACCESS_KEY secretkey
format as csv 
IGNOREHEADER 1;

#### Spark Pre Model
the following code creates temporary tables from given data files and the gnerated dictionary files.


In [ ]:
tempratureDf = spark.read.csv('../../GlobalLandTemperaturesByCity.csv',header=True, inferSchema=True) .where(year('dt')==2013).limit(1000)
tempratureDf.printSchema()
tempratureDf.createOrReplaceTempView("tempratureraw")
tempratureDf.show()

fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigrationDf=spark.read.parquet(fname)

immigrationDf.createOrReplaceTempView("immigrationraw")
immigrationDf.write.parquet("immigration2")
immigrationDf = immigrationDf.withColumn('arrdate2', expr("date_add(to_date('1960-01-01'),arrdate)"))
immigrationDf = immigrationDf.withColumn('depdate2', expr("date_add(to_date('1960-01-01'),depdate)"))
immigrationDf = immigrationDf.withColumn('dtadfile2', expr("to_date(dtadfile ,'yyyyMMdd')"))
immigrationDf = immigrationDf.withColumn('dtaddto2', expr("to_date(dtaddto,'MMddyyyy')"))
immigrationDf.printSchema()
immigrationDf.show()

airportsDf = spark.read.csv("airport-codes_csv.csv", header=True, inferSchema=True)
airportsDf.printSchema()
airportsDf.createOrReplaceTempView('airportsraw')
airportsDf.show()

portsDf = spark.read.csv("ports.csv", header=True, inferSchema=True)
portsDf.printSchema()
portsDf.createOrReplaceTempView('portsraw')
portsDf.show()

citiesDf=spark.read.csv("us-cities-demographics.csv",sep=';',header=True, inferSchema=True)
citiesDf.printSchema()
citiesDf.createOrReplaceTempView('citiesraw')
citiesDf.show()

statesDf = spark.read.csv("states.csv",  header=True, inferSchema=True)
statesDf.printSchema()
statesDf.createOrReplaceTempView('statesraw')
statesDf.show()

countriesDf = spark.read.csv("countries.csv", header=True, inferSchema=True)
countriesDf.printSchema()
countriesDf.createOrReplaceTempView('countriesraw')
countriesDf.show()

visamodelDf = spark.read.csv("VisaModel.csv",header=True, inferSchema=True)
visamodelDf.printSchema()
visamodelDf.createOrReplaceTempView('visamodelraw')
visamodelDf.show()

visaDf = spark.read.csv("Visa.csv", header=True, inferSchema=True)
visaDf.printSchema()
visaDf.createOrReplaceTempView('visaraw')
visaDf.show()

In [ ]:
#### 3.1 Conceptual Data Model

I generated Star Data Model from the raw data sets.

it''s most suitable model for our case.
apparently there are 4 main business entities here
1. Immigrants data.
2. Visa transactions
3. cities in united states.
4. temprature data  

so the data model will consist of 3 dimensions
1. temprature
2. city
3. immigrants

and one facts table visa_transactions

Hence we are working with 2 methodoligies in parallel, the following section will explain both of them:

##### Spark Conceptual Data Model.
we will work with this model in generated temporary views.
##### Redshift Conceptual Data Model
in redshift, we will have similar data model, except that table optimization is considered for performance improvement.
so tables cities is considered to be All diststyle. because it's limited data set 


In [ ]:
create table if not exists temprature(
    dt timestamp NOT NULL,
    AverageTemperature int4 NOT NULL,
    AverageTemperatureUncertainty varchar(256),
    cityid int ,
    primary key(dt, cityid) distkey
 );
   
create table if not exists city
(
  cityid int primary key,
  City  varchar(256) ,
  State  varchar(256) ,
  Median_Age  real ,
  Male_Population  integer ,
  Female_Population  integer ,
  Total_Population  integer ,
  Number_of_Veterans  integer ,
  Foreign_born  integer ,
  Average_Household_Size  real ,
  State_Code  varchar(256) ,
  White  int ,
  Asian int ,
  Hispanic_or_Latino int,
  Black_or_African_American int ,
  American_Indian_and_Alaska_Native int
) diststyle all;

create table if not exists immigrants
(
  id  int primary key distkey,  
  citizen  varchar(256) ,
  resource  varchar(256) ,
  addr  varchar(256) ,
  age  real ,
  count  real ,
  occup  varchar(256) ,
  birthyear  int ,
  gender  varchar(256) 
);
create table if not exists visa_transactions(
  id  int primary key distkey,
  year  int ,
  month  int ,
  port  varchar(256) ,
  arrdate  date ,
  model  varchar(256) ,
  depdate  date ,
  visa  varchar(256) ,
  visapost  varchar(256) ,
  entdepa  varchar(256) ,
  entdepd  varchar(256) ,
  entdepu  varchar(256) ,
  matflag  varchar(256) ,
  dtaddto  varchar(256) ,
  insnum  varchar(256)  ,
  airline  varchar(256) ,
  admnum  real ,
  fltno  varchar(256) ,
  visatype  varchar(256) 
);

#### 3.2 Mapping Out Data Pipelines

after we loaded data successfully into raw tables; using Spark data frame or from S3 to Redshift, remaining task is quite straight forward, we can simply load data by joining necessary raw tables with dictionaries as explained in the following section.

this graph shows the resulting star model
<img src="graphs/StarDataModel.png">

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
the following are the main queries required to 

In [ ]:
select i.cicid id ,i.i94yr year,i.i94mon month,i.i94port port,i.arrdate,vm.type model,i.depdate,v.type visa,i.visapost,i.entdepa,i.entdepd ,i.entdepu ,i.matflag ,i.dtaddto ,i.insnum ,i.airline ,i.admnum ,i.fltno ,i.visatype 
    from immigrationraw i join visamodeldic vm on i.i94mode =vm.code  
                          join visadic v on i.i94visa = v.code

In [ ]:
select i.cicid id ,c.country citizen, c2.country resource,s.state addr,i.i94bir age ,i.count , i.occup ,i.biryear birthyear, i.gender  
  from immigrationraw i join countriesdic c on i.i94cit =c.code
  join countriesdic c2 on i.i94res =c2.code
  join statesdic s on i.i94addr = s.code

In [ ]:
select a.City , a.State, a.`Median Age` Median_Age , a.`Male Population` Male_Population , a.`Female Population` Female_Population, 
         a.`Total Population` Total_Population, a.`Number of Veterans` Number_of_Veterans, a.`Foreign-born` Foreign_born, a.`Average Household Size` Average_Household_Size , 
         a.`State Code` State_Code,a.count 
         ,nvl(b.count,0) ,nvl(c.count,0),nvl(d.count,0),nvl(e.count,0) 
  from citiesraw a left join citiesraw b on a.City=b.City and a.State =b.State 
                left join citiesraw c on a.City=c.City and a.State =c.State
                left join citiesraw d on a.City=d.City and a.State =d.State
                left join citiesraw e on a.City=e.City and a.State =e.State
             where a.Race='White' and b.Race='Asian' and c.Race='Hispanic or Latino' and d.Race='Black or African-American' and e.Race='American Indian and Alaska Native'


In [ ]:
# Write code here
visatransactions=spark.sql(''' select i.cicid id ,i.i94yr year,i.i94mon month,i.i94port port,i.arrdate,vm.type model,i.depdate,v.type visa,i.visapost,i.entdepa,i.entdepd ,i.entdepu ,i.matflag ,i.dtaddto ,i.insnum ,i.airline ,i.admnum ,i.fltno ,i.visatype 
from immigrationraw i join visamodeldic vm on i.i94mode =vm.code  join visadic v on i.i94visa = v.code ''')
visatransactions.show()
visatransactions.createOrReplaceTempView("visatransactions")

immigrants=spark.sql('''select i.cicid id ,c.country citizen, c2.country resource,s.state addr,i.i94bir age ,i.count , i.occup ,i.biryear birthyear, i.gender  
          from immigrationraw i join countriesdic c on i.i94cit =c.code
          join countriesdic c2 on i.i94res =c2.code
          join statesdic s on i.i94addr = s.code''')
immigrants.show()
immigrants.createOrReplaceTempView("immigrants")

cities=spark.sql(''' select a.City , a.State, a.`Median Age` Median_Age , a.`Male Population` Male_Population , a.`Female Population` Female_Population, 
     a.`Total Population` Total_Population, a.`Number of Veterans` Number_of_Veterans, a.`Foreign-born` Foreign_born, a.`Average Household Size` Average_Household_Size , 
     a.`State Code` State_Code,a.count 
     ,nvl(b.count,0) ,nvl(c.count,0),nvl(d.count,0),nvl(e.count,0) 
      from citiesraw a left join citiesraw b on a.City=b.City and a.State =b.State 
                    left join citiesraw c on a.City=c.City and a.State =c.State
                    left join citiesraw d on a.City=d.City and a.State =d.State
                    left join citiesraw e on a.City=e.City and a.State =e.State
                 where a.Race='White' and b.Race='Asian' and c.Race='Hispanic or Latino' and d.Race='Black or African-American' and e.Race='American Indian and Alaska Native'
        ''')
cities.show()
cities.createOrReplaceTempView("cities")

#### 4.2 Data Quality Checks


In [ ]:
# Perform quality checks here
spark.sql(''' select count(*) from immigrationraw''').show()
spark.sql('''select count(distinct City) from citiesraw''').show()

spark.sql(''' select count(*) from visatransactions''').show()
spark.sql(''' select count(*) from immigrants''').show()
spark.sql(''' select count(*) from cities''').show()

* number of records in immigration raw data set. : 3,096,313
* number of records in visatransactions fact table is : 3,096,074

difference is due to some missing data in i94mod and i94visa
which can be got by this query
```
spark.sql(''' select count(*) from immigrationraw where i94mode is null or i94visa is null''').show()
```
resulting in 239

but much bigger discrepancy found between number of immigrants and original raw immigration data set, because a lot of nulls in i94addr which is the state where the immigrant will reside in

```
spark.sql(''' select count(*) from immigrationraw where i94res is null or i94cit is null or i94addr is null''').show()
```



#### 4.3 Data dictionary 
|table_name|column_name|?column?|?column?|
|----------|-----------|--------|--------|
|temprature|dt|tempratureraw|time stamp of temprature reading|
|temprature|averagetemperature|tempratureraw| average temprate read|
|temprature|averagetemperatureuncertainty|tempratureraw|certainty of temprature sensor accuracy|
|temprature|cityid|tempratureraw| city where the temprature was measured|
| | | | |
|city|cityid|citiesraw| city id |
|city|city|citiesraw|city name |
|city|state|citiesraw| state of city|
|city|median_age|citiesraw| median age of population in city|
|city|male_population|citiesraw| number of males in city|
|city|female_population|citiesraw| number of females in city|
|city|total_population|citiesraw| total number of population in city|
|city|number_of_veterans|citiesraw| number of experienced persons in city|
|city|foreign_born|citiesraw| number of persons born outside USA|
|city|average_household_size|citiesraw| the average number of family|
|city|state_code|citiesraw| sate code for the sate city belongs to|
|city|white|citiesraw| number of population of white race in city|
|city|asian|citiesraw|number of population of asian race in city|
|city|hispanic_or_latino|citiesraw|number of population of hispanic or latino race in city|
|city|black_or_african_american|citiesraw|number of population of black or african american race in city|
|city|american_indian_and_alaska_native|citiesraw|number of population of american indian or alaska native race in city|
| | | | |
|immigrants|id|immigrationraw| id for immigrant|
|immigrants|citizen|countriesraw| immigrant citizenship, the equivalent country name for code in immigrationraw.i94cit|
|immigrants|resource|immigrationraw|immigrant country when applied for visa, the equivalent country name for code in immigrationraw.i94res|
|immigrants|addr|immigrationraw| state code|
|immigrants|age|immigrationraw| age of immigrant|
|immigrants|count|immigrationraw| number of visas|
|immigrants|occup|immigrationraw| occupation of the immigrant|
|immigrants|birthyear|immigrationraw| birth year of the immigrant|
|immigrants|gender|immigrationraw|gender of immigrant|
| | | | |
|visa_transactions|id|immigrationraw| id for visa transaction|
|visa_transactions|year|immigrationraw| year of crossing border |
|visa_transactions|month|immigrationraw| month of crossing border|
|visa_transactions|port|immigrationraw| port of entrance|
|visa_transactions|arrdate|immigrationraw| arrival date|
|visa_transactions|model|immigrationraw| visa model air,sea,land, other|
|visa_transactions|depdate|immigrationraw| leaving of country|
|visa_transactions|visa|immigrationraw| visa type , busines or student or pleasure |
|visa_transactions|visapost|immigrationraw| visa admission office in foreign country|
|visa_transactions|entdepa|immigrationraw|Arrival Flag - admitted or paroled into the U.S.|
|visa_transactions|entdepd|immigrationraw|Departure Flag - Departed, lost I-94 or is deceased|
|visa_transactions|entdepu|immigrationraw|Update Flag - Either apprehended, overstayed, adjusted to perm residence|
|visa_transactions|matflag|immigrationraw|Match flag - Match of arrival and departure records |
|visa_transactions|dtaddto|immigrationraw| Character Date Field - Date to which admitted to U.S. (allowed to stay until)|
|visa_transactions|insnum|immigrationraw| INS number|
|visa_transactions|airline|immigrationraw| airline name of immigrant flight|
|visa_transactions|admnum|immigrationraw| admission serian number|
|visa_transactions|fltno|immigrationraw| flight number |
|visa_transactions|visatype|immigrationraw| visa category |


#### Step 5: Complete Project Write Up
## Goal of project
final output from this data engineering project was to clean and transform data into analytical model that is it's ready to be analyzed with sql queries to extract patterns and insights by data science team.

we showed 2 methodologies for loading data: -

with Spark Data frames and SQL, resulting model was tables those can be queries with Spark sql.
with S3 and redshift, resulting similar star model in redshift.

## Queries could be run.

## Tools used.
* spark data frames and spark sql
* S3 and redshift
* (optional) Airflow

given data set is considered small (about 6 Giga bytes) and almost fully structured.
6GB can fit with single machine memory , so using Spark data frames and spark sql will achieve the goal of project.

to use cloud technologies, we could store data in S3 buckets and process it using Redshift cluster.
redshift gives us a lot of flexibility and better performance especialy with using correct optimization when creating dimensions and Fact tables.

Airflow could be used to schedule loading data from raw datasets to data ware house model.
not a lot of operators required, mainly S3 to Redshift operator.
other postgres operators may be needed to run queries for reporting or dashboard.

## Model choice
there are many alternatives for data modeling in our project, we could choose either star data model or snow-flake model.
snow-flake is more relevant espacially with the hierarchy of demographic data such as {airport >> city >> state }
but it will be more distracting especially there is no more data for states for example, so to make it tidy model, star data model was adopted in project.

it's clear that few business entities are embedded in given raw data sets, as discussed before.
dictionary data are used directly to interpret ambigous columns instead of doing many references.

#### Data update schedule.
about 3 million visa transactions happens per year, so, it will be a good practice to load this data daily.
one could suggest best hour for loading data with least load, but dates included contains no information about the exact time of transaction.

#### Github repository 
https://github.com/engomar2003/UdacityDataEngineeringCapstoneProject.git


#### Changes required on solution if data changed by the following factors:
* **The data was increased by 100x.**

     current data size is about 6GB for immigration dataset. 0.5 GB for tempratures data set, few MBs for other data sets.
     if the data increases 100 times (approaching 1 Tera), reserving big space in Redshift cluster will be expensive,
     also dealing with it using Spark SQL will be over-utilizing memories and may lead to memory overflow crash.
     
     best solution is using EMR (Spark +S3) or EMR (serverless).
     in this case we use S3 as reliable cheap storage, and with Spark or lambdas , we can perform the needed processing.
     
     also note: S3 bucket doesn't allow uploading files over 160GB in direct way, so it will suggest using:
     **AWS CLI, AWS SDK, or Amazon S3 REST API**
     or Airflow should be used for data loading and transformation, with partitioned data by month, to increase performance.
     
     
* **The data populates a dashboard that must be updated on a daily basis by 7am every day.**

     in this case we use Airflow to load data and execute needed queries for dashboard per agreed schedule.
     or we can use EMR serverless, and define schedule for running lambdas.
     in both cases, it's better to use S3 for storage as the data is required to be processed only at short time.
     this will be more efficient. money-wise.
     
* **The database needed to be accessed by 100+ people.**

     in this case, data will be accessed a lot and over a long period of time.
     best practice will be using Redshift cluster or EMR with HDFS+Spark (if larger size)   

